In [1]:
!pip install -q kaggle

In [2]:
from google.colab import files

In [8]:
# get kaggle api key from kaggle-->Account-->API section and upload it here.

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"teddwebber","key":"cbb9f79ca52e65c87cfcb88aaea2b210"}'}

In [10]:
# Creates hidden folder in root directory and saves kaggle.json in it.

! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 /root/.kaggle/kaggle.json

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [11]:
!kaggle datasets download -d ealaxi/paysim1
!unzip /content/paysim1.zip

100% 177M/178M [00:06<00:00, 36.9MB/s]
100% 178M/178M [00:06<00:00, 29.0MB/s]
Archive:  /content/paysim1.zip
  inflating: PS_20174392719_1491204439457_log.csv  


In [12]:
import pandas as pd
import numpy as np

df = pd.read_csv("PS_20174392719_1491204439457_log.csv")
pd.set_option("display.max.columns", None)
df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


In [13]:
len_not_fraud = len(df['isFraud'][df.isFraud == 0])
len_is_fraud = len(df['isFraud'][df.isFraud == 1])

arr = np.array([len_not_fraud, len_is_fraud])
labels = ['Not Fraudulent', 'Fraudulent']
print(f"Total No. of Non-Fraudulent Cases: {len_not_fraud}")
print(f"Total No. Fraudulent Cases: {len_is_fraud}")

Total No. of Non-Fraudulent Cases: 6354407
Total No. Fraudulent Cases: 8213
